In [ ]:
import itertools
import functools
import random
import collections
import operator
import sys
import re
import copy
import numpy as np
import math
import networkx as nx

In [ ]:
from helpers.functions import *

Configuration

In [ ]:
DIR = "data/2019/"
load_day = functools.partial(load, DIR)

# Intcode

In [ ]:
class IntCode():
    
    def __init__(self, instructions, inputs=None, relative_base=0):
        
        self.memory = collections.defaultdict(int)
        self.memory.update(dict(enumerate(instructions)))
        self.inputs = inputs or []
        self.relative_base = relative_base
        self.ptr = 0
        
    def value(self, ptr, mode):
        if mode == 0:
            return self.memory[ptr]
        elif mode == 1:
            return ptr
        elif mode == 2:
            return self.relative_base + self.memory[ptr]
        else:
            raise ValueError("Unkown mode {}".format(mode))
    
    def get_instruction(self, ptr):
        opcode = self.memory[ptr] % 100 
        param1_mode = (self.memory[ptr] % 1000) // 100
        param2_mode = (self.memory[ptr] % 10000) // 1000
        param3_mode = (self.memory[ptr] % 100000) // 10000
        return opcode, param1_mode, param2_mode, param3_mode   
        
        
    def get_last_output(self):
        final_output = None
        for output in self.run():
            final_output = output
            
        return final_output
            
    def run(self):
        
        while self.memory[self.ptr] != 99:
            opcode, pm1, pm2, pm3 = self.get_instruction(self.ptr)
    
            a = self.value(self.ptr + 1, pm1)
            b = self.value(self.ptr + 2, pm2)
            c = self.value(self.ptr + 3, pm3)
        
            if opcode == 1:
                self.memory[c] = self.memory[a] + self.memory[b]
                self.ptr += 4

            elif opcode == 2:
                self.memory[c] = self.memory[a] * self.memory[b]
                self.ptr += 4
                
            elif opcode == 3:
                if not self.inputs:
                    break
                
                self.memory[a] = self.inputs.pop(0)
                self.ptr += 2
                
            elif opcode == 4:
                self.ptr += 2
                yield self.memory[a]
                
            elif opcode == 5:
                self.ptr = self.memory[b] if self.memory[a] != 0 else self.ptr+3

            elif opcode == 6:
                self.ptr = self.memory[b] if self.memory[a] == 0 else self.ptr+3

            elif opcode == 7:
                self.memory[c] = 1 if self.memory[a] < self.memory[b] else 0
                self.ptr += 4

            elif opcode == 8:
                self.memory[c] = 1 if self.memory[a] == self.memory[b] else 0
                self.ptr += 4
                
            elif opcode == 9:
                self.relative_base += self.memory[a]
                self.ptr += 2
            
            elif opcode == 99:
                break
                
            else:
                raise ValueError("Unkown opcode {}".format(opcode))

# Problems

## Day 1

http://adventofcode.com/2019/day/1

In [ ]:
content = mapl(int, load_day(1))

__Part 1__

In [ ]:
def get_fuel(mass):
    return max(0, math.floor(mass/3) - 2)

In [ ]:
sum(map(get_fuel, content))

__Part 2__

In [ ]:
def get_over_all_fuel(mass):
    if mass <= 0:
        return 0
    
    fuel = get_fuel(mass)
    return fuel + get_over_all_fuel(fuel)

In [ ]:
sum(map(get_over_all_fuel, content))

## Day 2

http://adventofcode.com/2019/day/2

In [ ]:
instructions = mapl(int, load_day(2)[0].split(","))

__Part 1__

In [ ]:
prog = IntCode(instructions, [1])
prog.memory[1] = 12
prog.memory[2] = 2
prog.get_last_output()

print(prog.memory[0])

__Part 2__

In [ ]:
for i in range(100):
    for j in range(100):
        prog = IntCode(instructions, [1])
        prog.memory[1] = i
        prog.memory[2] = j
        prog.get_last_output()
        if prog.memory[0] == 19690720:
            print(100*i+j)

## Day 3

http://adventofcode.com/2019/day/3

In [ ]:
content = load_day(3)

In [ ]:
path1 = content[0].split(",")
path2 = content[1].split(",")

In [ ]:
directions = {
    "R": (1,0), 
    "L": (-1,0),
    "U": (0,1),
    "D": (0,-1),
}

def get_positions(path):
    positions = list()
    current_position = (0,0)
    for step in path:
        direction, number = step[0], int(step[1:])
        for i in range(number):
            current_position = tuple(add(current_position, directions[direction]))
            positions.append(current_position)
            
    return positions

In [ ]:
positions_path1 = get_positions(path1)
positions_path2 = get_positions(path2)

In [ ]:
commun = set(positions_path1).intersection(set(positions_path2))

__Part 1__

In [ ]:
min([abs(x)+abs(y) for x, y in commun])

__Part 2__

In [ ]:
steps = []
for c in commun:
    idx1 = positions_path1.index(c) + 1
    idx2 = positions_path2.index(c) + 1
    steps.append(idx1 + idx2)

In [ ]:
min(steps)

## Day 4

http://adventofcode.com/2019/day/4

In [ ]:
low = 356261
high = 846303

In [ ]:
def never_decrease(numbers):
    return all(x <= y for x,y in zip(numbers, numbers[1:]))

def contains_a_pair(numbers):
    return any(x == y for x,y in zip(numbers, numbers[1:]))

In [ ]:
pwds = []
for pwd in range(low, high+1):
    numbers = mapl(int, str(pwd))
    if never_decrease(numbers) and contains_a_pair(numbers):
        pwds.append(numbers)

__Part 1__

In [ ]:
len(pwds)

__Part 2__

In [ ]:
sum(1 for pwd in pwds if 2 in collections.Counter(pwd).values())

## Day 5

http://adventofcode.com/2019/day/5

In [ ]:
instructions = mapl(int, load_day(5)[0].split(","))

__Part 1__

In [ ]:
prog = IntCode(instructions, [1])
prog.get_last_output()

__Part 2__

In [ ]:
prog = IntCode(instructions, [5])
prog.get_last_output()

## Day 6

http://adventofcode.com/2019/day/6

In [ ]:
content = mapl(lambda x: x.split(")"), load_day(6))

In [ ]:
parents = dict()
childrens = collections.defaultdict(set)

for u,v in content:
    parents[v] = u
    childrens[u].add(v)

__Part 1__

In [ ]:
total = 0

for obj in parents.keys():
    while obj != "COM":
        total += 1
        obj = parents[obj]
        
total

__Part 2__

In [ ]:
start = "YOU"
end = "SAN"

explored = set()

def explore(source, root, transfers):
    explored.add(root)
    
    if root == "SAN":
        return transfers
    
    nodes = childrens[root]
    if root in parents: 
        nodes.add(parents[root])
    
    for node in nodes:
        if node not in explored:
            out = explore(root, node, transfers+1)
            if out is not None:
                return out

In [ ]:
explore(start, parents[start], 0) - 1

## Day 7

http://adventofcode.com/2019/day/7

In [ ]:
instructions = mapl(int, load_day(7)[0].split(","))

__Part 1__

In [ ]:
max_amp = 0
for phases in itertools.permutations(range(5)):
    amp = 0
    for p in phases:
        amp = IntCode(instructions, [p, amp]).get_last_output()

    max_amp = max(max_amp, amp)

In [ ]:
max_amp

__Part 2__

    #TODO
    max_amp = 0
    for phases in itertools.permutations(range(5,10)):
        amp_instructions = [instructions.copy() for _ in range(5)]
        amp_inputs = [[p] for p in phases]
        amp_ptr = [0, 0, 0, 0, 0]

        amp = 0

        while amp is not None:
            for i in range(5):
                amp_inputs[i].append(amp)
                amp, amp_ptr[i] = Int(amp_instructions[i], amp_inputs[i], ptr=amp_ptr[i])

            last_amp = last_amp if amp is None else amp

        max_amp = max(max_amp, last_amp)

    max_amp

## Day 8

http://adventofcode.com/2019/day/8

In [ ]:
content = load_day(8)[0]

In [ ]:
width = 25
height = 6
layers_count = (len(content) // width) // height

In [ ]:
pixels = mapl(int, content)
layers = np.zeros((layers_count, height, width))

for l in range(layers_count):   
    for y in range(height):
        for x in range(width):
            layers[l, y, x] = int(pixels.pop(0))

__Part 1__

In [ ]:
min_zeros = sys.maxsize
output = None
for l in range(layers_count):
    counter = collections.Counter(layers[l].flatten())
    if counter[0] < min_zeros:
        output = counter[1] * counter[2]
        min_zeros = counter[0]

In [ ]:
output

__Part 2__

In [ ]:
for y in range(height):
    for x in range(width):
        val = next(e for e in np.array(layers)[:, y, x] if e != 2)
        
        if val == 2:
            print("?", end="")
        elif val == 1:
            print("#", end="")
        elif val == 0:
            print(" ", end="")
    print()

## Day 9

http://adventofcode.com/2019/day/9

In [ ]:
instructions = mapl(int, load_day(9)[0].split(","))

__Part 1__

In [ ]:
prog = IntCode(instructions, [1])
prog.get_last_output()

__Part 2__

In [ ]:
prog = IntCode(instructions, [2])
prog.get_last_output()

## Day 10

http://adventofcode.com/2019/day/10

In [ ]:
asteroids_belt = np.array(mapl(list, load_day(10)))

In [ ]:
asteroids = set()
for x in range(asteroids_belt.shape[0]):
    for y in range(asteroids_belt.shape[1]):
        if asteroids_belt[x, y] == "#":
            asteroids.add((y, x))

In [ ]:
stations2neighbors = collections.defaultdict(set)

for sy, sx in asteroids:
    for ay, ax in asteroids:
        if (sy, sx) == (ay, ax):
            continue
            
        y, x = (ay - sy, ax - sx)
        gcd = abs(math.gcd(x, y))
        stations2neighbors[(sy, sx)].add((y // gcd, x // gcd))

__Part 1__

In [ ]:
max_value, station = max((len(x), pos) for pos, x in stations2neighbors.items())

In [ ]:
max_value, station

__Part 2__

In [ ]:
to_zap = sorted(((math.atan2(dy, dx), (dy, dx)) for dy, dx in stations2neighbors[station]), reverse=True)

_, (dy, dx) = to_zap[200-1]

y, x = station[0] + dy, station[1] + dx
while (x, y) not in asteroids:
    x, y = x + dx, y + dy

In [ ]:
y*100 + x

## Day 11

http://adventofcode.com/2019/day/11

In [ ]:
instructions = mapl(int, load_day(11)[0].split(","))

In [ ]:
DIRECTIONS = [(0, -1), (1, 0), (0, 1), (-1, 0)]

In [ ]:
def paint(instructions, start_color=0):
    x, y = 0, 0
    direction = 0

    panels = collections.defaultdict(int)
    panels[(x,y)] = start_color
    
    prog = IntCode(instructions, [panels[(x,y)]])
    execute = prog.run()

    for output1, output2 in zip(execute, execute):
        panels[(x,y)] = output1
        direction = ((direction + 1) if output2 == 1 else (direction - 1 + len(DIRECTIONS))) % len(DIRECTIONS)
        x,y = x + DIRECTIONS[direction][0], y + DIRECTIONS[direction][1]
        prog.inputs.append(panels[(x,y)])
        
    return panels

__Part 1__

In [ ]:
panels = paint(instructions, start_color=0)

In [ ]:
len(panels)

__Part 2__

In [ ]:
panels = paint(instructions, start_color=1)

xs = [x for x, y in panels]
ys = [y for x, y in panels]

for y in range(min(ys), max(ys)+1):
    for x in range(min(xs), max(xs)+1):

        print("#" if panels[(x, y)] else " ", end="")
        
    print()

## Day 12

http://adventofcode.com/2019/day/12

In [ ]:
content = load_day(12)

In [ ]:
planets = np.array([extract_ints(line) for line in content])
combinations = np.array(list(itertools.combinations(range(len(planets)), 2)))

In [ ]:
base_state = np.zeros((len(planets), 6), dtype=int)
base_state[:, :3] = planets

__Part 1__

In [ ]:
state = base_state.copy()

for _ in range(1000):
    np.add.at(state[:, 3:], combinations, np.sign(state[combinations, :3][:,[1, 0]] - state[combinations, :3][:,[0, 1]]))
    state[:, :3] += state[:, 3:]

(np.abs(state[:, :3]).sum(axis=1) * np.abs(state[:, 3:]).sum(axis=1)).sum()

__Part 2__

In [ ]:
state = base_state.copy()

In [ ]:
#TODO

## Day 13

http://adventofcode.com/2019/day/13

In [ ]:
instructions = mapl(int, load_day(13)[0].split(","))

__Part 1__

In [ ]:
prog = IntCode(instructions)

In [ ]:
outputs = [out for out in prog.run()]

In [ ]:
collections.Counter(outputs[2:][::3])[2]

__Part 2__

In [ ]:
prog = IntCode(instructions)
prog.memory[0] = 2

move = 0
score = 0
ball = (0,0)
paddle = (0,0)

pixels = collections.defaultdict(int)

iteration = 0
while True:
    prog.inputs.append(move)
    outputs = [out for out in prog.run()]
 
    while outputs:
        x = outputs.pop(0)
        y = outputs.pop(0)
        tile_id = outputs.pop(0)

        if (x, y) == (-1, 0):
            score = tile_id
        else:
            pixels[(x, y)] = tile_id

            if tile_id == 4:
                ball = (x, y)
            elif tile_id == 3:
                paddle = (x, y)
            
    if ball[0] < paddle[0]:
        move = -1
    elif ball[0] > paddle[0]:
        move = 1
    else:
        move = 0
        
    if collections.Counter(pixels.values()).get(2, 0) == 0:
        break

In [ ]:
score

## Day 14

http://adventofcode.com/2019/day/14

In [ ]:
content = load_day(14)

In [ ]:
reactions = {}
for line in content:
    *inputs, (out_qty, out_elem) = re.findall(r'(\d+) (\w+)', line)
    reactions[out_elem] = (int(out_qty), mapl(lambda x: (int(x[0]), x[1]), inputs))

In [ ]:
def get_required_ore(fuel_amount):

    needed = {
        'FUEL': fuel_amount
    }

    def get_requirements():
        return [elem for elem, qty in needed.items() if qty > 0 and elem != 'ORE']

    while get_requirements():
        for element in get_requirements():
            qty, inputs = reactions[element]
            scaling = (needed[element] + qty - 1) // qty

            for (src_qty, src_elem) in inputs:
                needed[src_elem] = needed.get(src_elem, 0) + scaling*src_qty

            needed[element] -= scaling*qty

    return needed['ORE']

__Part 1__

In [ ]:
get_required_ore(1)

__Part 2__

In [ ]:
ore_low  = 1
ore_high = 10
ore_amount = 10e11

while get_required_ore(ore_high) <= ore_amount:
    ore_high *= 10
    
# Use binary search
while ore_high - ore_low > 1:
    ore_middle = (ore_high + ore_low) // 2
    if get_required_ore(ore_middle) <= ore_amount:
        ore_low = ore_middle
    else:
        ore_high = ore_middle

In [ ]:
print(ore_low)

## Day 15

http://adventofcode.com/2019/day/15

In [ ]:
instructions = mapl(int, load_day(15)[0].split(","))

In [ ]:
MOVES = [(1, 0), (-1, 0), (0, -1), (0, 1)]

In [ ]:
prog = IntCode(instructions)
execute = prog.run()
oxygen = None
position = (0, 0)
locations = set()

while not oxygen:
    direction = random.randint(1, 4)
    prog.inputs.append(direction)
    status = next(execute)

    if status == 0:
        continue
    elif status == 2:
        oxygen = position

    locations.add(position)
    position = tuple(add(position, MOVES[direction - 1]))

In [ ]:
G = nx.Graph()
G.add_node((0, 0))

for (x1, y1) in locations:
    for (x2, y2) in MOVES:
        position = (x1 + x2, y1 + y2)
        if position in locations:
            G.add_edge((x1, y1), position)

__Part 1__

In [ ]:
len(nx.shortest_path(G, (0, 0), oxygen))

__Part 2__

In [ ]:
#TODO

## Day 16

http://adventofcode.com/2019/day/16

In [ ]:
content = mapl(int, list(load_day(16)[0]))

BASE_PATTERN = [0, 1, 0, -1]

__Part 1__

In [ ]:
for phase in range(100):
    for pos in range(len(content)):
        pattern = [p for p in BASE_PATTERN for _ in range(pos+1)]
        pattern.rotate(-1)
        
        res = 0
        for i in range(len(content)):
            res += content[i] * pattern[0]
            pattern.rotate(-1)
        
        content[pos] = abs(res) % 10

In [ ]:
"".join(map(str, content[:8]))

__Part 2__

In [ ]:
input    = read_file("16")[0]
offset   = int(input[:7])
elements = [int(num) for _ in range(10000) for num in input][offset:]

for _ in range(100):
    for i in range(-2, -len(elements)-1, -1):
        elements[i] = (elements[i] + elements[i+1]) % 10

return "".join([str(x) for x in elements[:8]])

## Day 17

http://adventofcode.com/2019/day/17

In [ ]:
instructions = mapl(int, load_day(17)[0].split(","))

__Part 1__

In [ ]:
prog = IntCode(instructions)
scaffolding = "".join(chr(o) for o in prog.run()).strip().split("\n")
height = len(scaffolding)
width = len(scaffolding[0])

In [ ]:
def is_intersection(x, y):
    if x <= 1 or y <= 1 or x+1 > width-1 or y+1 > height-1:
        return False
    
    return (scaffolding[y][x] == "#" 
            and scaffolding[y][x+1] == "#" 
            and scaffolding[y][x-1] == "#" 
            and scaffolding[y+1][x] == "#"
            and scaffolding[y-1][x] == "#")

In [ ]:
crossing = []
for y in range(height):
    for x in range(width-1):
        if is_intersection(x, y):
            crossing.append((x*y))

In [ ]:
sum(crossing)

__Part 2__

## Day 18

http://adventofcode.com/2019/day/18

__Part 1__

__Part 2__

## Day 19

http://adventofcode.com/2019/day/19

In [ ]:
instructions = mapl(int, load_day(19)[0].split(","))

In [ ]:
values = {}

def get_beam_value(x, y):
    if (x, y) not in values:
        values[(x, y)] = IntCode(instructions, [x, y]).get_last_output()
    return values[(x, y)] 

__Part 1__

In [ ]:
sum(get_beam_value(x, y) for x in range(50) for y in range(50))

__Part 2__

In [ ]:
x = 0
for y in range(100, 100000):
    while not get_beam_value(x, y):
        x += 1
    if get_beam_value(x+99, y-99):
        break

In [ ]:
 10000*x + (y-99)

## Day 20

http://adventofcode.com/2019/day/20

In [ ]:
content = load_day(20)

__Part 1__

__Part 2__

## Day 21

http://adventofcode.com/2019/day/21

In [ ]:
instructions = mapl(int, load_day(21)[0].split(","))

__Part 1__

In [ ]:
commands = ["NOT A J", "NOT B T", "OR T J", "NOT C T", "OR T J", "NOT D T", "NOT T T", "AND T J", "WALK"]
inputs = mapl(ord, "\n".join(commands) + "\n")
prog = IntCode(instructions, inputs)
out = [chr(x) if x < 127 else x for x in prog.run()]
print(''.join(map(str, out)))

__Part 2__

In [ ]:
commands = ["NOT A J", "NOT B T", "OR T J", "NOT C T", "OR T J", "NOT D T", "NOT T T", "AND T J", "NOT I T", "NOT T T", "OR F T", "AND E T", "OR H T", "AND T J", "RUN"]
inputs = mapl(ord, "\n".join(commands) + "\n")
prog = IntCode(instructions, inputs)
out = [chr(x) if x < 127 else x for x in prog.run()]
print(''.join(map(str, out)))

## Day 22

http://adventofcode.com/2019/day/22

In [ ]:
content = load_day(22)

In [ ]:

n = 10007
c = 2019

def deal_new(c, n):    return (-c - 1) % n
def deal_inc(c, n, i): return ( c * i) % n
def cut(c, n, i):      return ( c - i) % n

for l in content:
    if l == 'deal into new stack\n':
        c = deal_new(c, n)
    elif l.startswith('deal with increment '):
        c = deal_inc(c, n, int(l[len('deal with increment '):]))
    elif l.startswith('cut '):
        c = cut(c, n, int(l[len('cut '):]))

print(c)

In [ ]:
from itertools import *
from math import gcd

shuffles = [x.strip().split() for x in content]
N = 10007
new_stack_i = lambda i: N-i-1
cut_i = lambda i, n:  (i-n)%N
deal_i = lambda i, n: (i*n)%N

def track_pos(i):
    for shuf in shuffles:
        if shuf[0]=="cut":
            i = cut_i(i, int(shuf[1]))
        elif shuf[0]=="deal" and shuf[2]=="increment":
            i = deal_i(i, int(shuf[-1]))
        elif shuf[0]=="deal" and shuf[-1]=="stack":
            i = new_stack_i(i)
    return i
print(f"Part 1: {track_pos(2019)}")

N, REP = 119315717514047, 101741582076661
def egcd(a, b):
    x,y, u,v = 0,1, 1,0
    while a != 0:
        q,r = b//a,b%a; m,n = x-u*q,y-v*q 
        b,a, x,y, u,v = a,r, u,v, m,n
    return b, x, y

def modinv(a, m):
    g, x, y = egcd(a, m) 
    return (None if g!=1 else x%m)

p0, p1 = track_pos(0), track_pos(1)
a1, b1 = (p1-p0)%N, p0
a_t = modinv(a1, N) 
b_t = (-a_t*b1)%N

poly = lambda x: pow(a_t, REP, N)*x + (pow(a_t, REP, N)-1) * modinv(a_t-1, N) * b_t
print(f"Part 2: {poly(2020)%N}")

In [ ]:
REVERSE_RE = r'deal into new stack'
CUT_RE = r'cut (-?\d+)'
INCREMENT_RE = r'deal with increment (\d+)'


@functools.lru_cache(maxsize=None)
def inverse(n, k):
    return numbers.mod_inverse(k, n)


def mod(n, *coeffs):
    return tuple(coeff % n for coeff in coeffs)


COEFFS = {
    REVERSE_RE: lambda n, a, b: mod(n, -1 * a, -1 * b - 1),
    CUT_RE: lambda n, k, a, b: mod(n, a, b - k),
    INCREMENT_RE: lambda n, k, a, b: mod(n, a * k, b * k),
}

INV_COEFFS = {
    REVERSE_RE: lambda n, a, b: mod(n, -1 * a, -1 * b - 1),
    CUT_RE: lambda n, k, a, b: mod(n, a, b + k),
    INCREMENT_RE: lambda n, k, a, b: mod(n, a * inverse(n, k), b * inverse(n, k)),
}


def get_coeffs(n, techniques, inverse=False):
    if inverse:
        techniques = techniques[::-1]
        coeff_map = INV_COEFFS
    else:
        coeff_map = COEFFS

    coeffs = (1, 0)
    for technique in techniques:
        for regex, method in coeff_map.items():
            match = re.match(regex, technique)
            if match:
                coeffs = method(*itertools.chain(
                    [n],
                    [int(arg) for arg in match.groups()],
                    coeffs,
                ))

    return coeffs


def shuffle(n, card, coeffs, rounds=1):
    a, b = coeffs
    return (
        pow(a, rounds, n) * card +
        b * (pow(a, rounds, n) - 1) * inverse(n, a - 1)
    ) % n


def cli():
    pass


def part_1():
    lines = load_day(22)
    n = 10007
    coeffs = get_coeffs(n, [line[0] for line in lines])
    print(shuffle(n, 2019, coeffs))

def part_2():
    lines = load_day(22)
    n = 119315717514047
    coeffs = get_coeffs(n, [line[0] for line in lines], inverse=True)
    print(shuffle(n, 2020, coeffs, rounds=101741582076661))



In [ ]:
part_1()

__Part 1__

__Part 2__

## Day 23

http://adventofcode.com/2019/day/23

In [ ]:
instructions = mapl(int, load_day(23)[0].split(","))

__Part 1__

In [ ]:
programs = []
for c in range(50):
    prog = IntCode(instructions, [c, -1])
    programs.append((prog, prog.run()))

----

In [ ]:
#!python3
import sys
from collections import deque, defaultdict


class IntCode(object):
    """ IntCode virtual machine """
    grid = defaultdict(str)

    def __init__(self, name, code, inp):
        self.name = name
        self.program_param = {}
        self.pos = 0
        self.output = 0
        self.program = {}
        self.io = deque()
        self.io.append(inp)
        self.relative_base = 0
        self.x = 50
        self.y = 50
        self.__direction = '^'

        i = 0
        for instr in code:
            self.program[i] = instr
            i += 1

    def __str__(self):
        return 1

    def get_input(self):
        """
        Gets input for the IntCode VM
        :param self.cur_proc: Which processor is now running
        (to enable multiple codes running at the same time)
        :return: the value of input
        """

#        print("Processor ",self.name," getting input")
        if len(self.io) > 0:
            instr = self.io.popleft()
        else:
            instr = -1
        return int(instr)

    def process_output(self):
        # if we have seen all address,
        # x, y, return it and delete from the queue
        if len(self.io) == 3:
            addr = self.io.popleft()
            x = self.io.popleft()
            y = self.io.popleft()
            return(addr, x, y)

        # otherwise keep running
        return (-1, -1, -1)

    def run_intcode(self):
        params = {1: 3, 2: 3, 3: 1, 4: 1, 5: 2, 6: 2, 7: 3, 8: 3, 9: 1, 99: 0}

        while 1:
            instr = self.program[self.pos]

            op = instr % 100

            mode1 = int(instr / 100) % 10
            mode2 = int(instr / 1000) % 10
            mode3 = int(instr / 10000) % 10

            reg1 = self.program.get(self.pos + 1, 0)
            reg2 = self.program.get(self.pos + 2, 0)
            reg3 = self.program.get(self.pos + 3, 0)

            v1 = reg1

            if op != 3:
                if mode1 == 0:
                    v1 = self.program.get(reg1, 0)
                if mode1 == 2:
                    v1 = self.program.get(
                            reg1 + self.relative_base, 0)
            else:
                if mode1 == 2:
                    v1 += self.relative_base

            v2 = reg2
            if mode2 == 0:
                v2 = self.program.get(reg2, 0)
            if mode2 == 2:
                v2 = self.program.get(
                        reg2 + self.relative_base, 0)

            v3 = reg3
            if mode3 == 2:
                v3 += self.relative_base

            if op == 1:
                self.program[v3] = v1 + v2

            # Process the opcodes
            elif op == 2:
                self.program[v3] = v1 * v2

            elif op == 3:
                self.program[v1] = self.get_input()
                if self.program[v1] == -1:
                    self.pos += 2
                    return(-1, -1, -1)

            elif op == 4:
                self.io.append(v1)
                (addr, x, y) = self.process_output()
                if addr != -1:
                    self.pos += 2
                    return (addr, x, y)

            elif (op == 5):
                if v1 > 0:
                    self.pos = v2
                    continue

            elif op == 6:
                if v1 == 0:
                    self.pos = v2
                    continue

            elif op == 7:
                if int(v1) < int(v2):
                    self.program[v3] = 1
                else:
                    self.program[v3] = 0

            elif op == 8:
                if v1 == v2:
                    self.program[v3] = 1
                else:
                    self.program[v3] = 0

            elif op == 9:
                self.relative_base += v1

            elif op == 99:
                return self.io

            else:
                sys.exit("Unknown argument found")

            shift = params[op] + 1
            self.pos = self.pos + shift


In [ ]:
import sys
from collections import deque


def is_network_idle():
    for i in range(50):
        if len(processors[i].io) > 0:
            return 0
    return 1


code = open("/Users/yoannponti/Projects/programming-exercises/advent-of-code/data/2019/day23.txt").read().strip().split(',')
data = list(map(int, code))

processors = deque()

for i in range(50):
    processor = IntCode(i, data, i)
    processors.append(processor)

i = 0
addr = 0

natx = naty = None
prev_naty = None
part = 1
empty_count = 0

while 1:
    (addr, x, y) = processors[i].run_intcode()

    if addr != -1 and addr != 255:
        processors[addr].io.append(x)
        processors[addr].io.append(y)
        empty_count = 0
        i = addr

    elif addr == 255:
        if part == 1:
            print("Part1 : ", y)
            part += 1

        natx = x
        naty = y
        empty_count = 0

    else:
        if is_network_idle() == 1:
            if empty_count == 10000:
                if prev_naty and naty == prev_naty:
                    print("Part2 : ", naty)
                    break

                prev_naty = naty

                processors[0].io.append(natx)
                processors[0].io.append(naty)
                empty_count = 0
            else:
                empty_count += 1
        i += 1
        i %= 50

__Part 2__

## Day 24

http://adventofcode.com/2019/day/24

In [ ]:
content = load_day(24)

__Part 1__

In [ ]:
import collections
import math
import re
import sys
def step(grid):
    grid2 = []
    def get(x, y):
        a = []
        if x > 0:
            a.append(grid[x-1][y])
        if x < len(grid)-1:
            a.append(grid[x+1][y])
        if y > 0:
            a.append(grid[x][y-1])
        if y < len(grid[0])-1:
            a.append(grid[x][y+1])
        return a
    for i in range(len(grid)):
        line2 = ''
        for j in range(len(grid[0])):
            if grid[i][j] == '#':
                ch = '#' if get(i,j).count('#') == 1 else '.'
            else:
                ch = '#' if get(i,j).count('#') in (1,2) else '.'
            line2 += ch
        grid2.append(line2)
    return grid2


def stepb(bugs):
    minlev = min(l for l, x, y in bugs)
    maxlev = max(l for l, x, y in bugs)

    def countadj(l, x, y):
        c = 0
        adj = [
            (l, x-1,y),
            (l, x+1,y),
            (l, x,y-1),
            (l, x,y+1),
        ]
        actadj = []
        for aa in adj:
            la, xa, ya = aa
            if xa == -1:
                actadj.append((la-1, 1, 2))
            elif xa == 2 and ya == 2 and x == 1:
                actadj.append((la+1, 0, 0))
                actadj.append((la+1, 0, 1))
                actadj.append((la+1, 0, 2))
                actadj.append((la+1, 0, 3))
                actadj.append((la+1, 0, 4))
            elif xa == 2 and ya == 2 and x == 3:
                actadj.append((la+1, 4, 0))
                actadj.append((la+1, 4, 1))
                actadj.append((la+1, 4, 2))
                actadj.append((la+1, 4, 3))
                actadj.append((la+1, 4, 4))
            elif xa == 5:
                actadj.append((la-1, 3, 2))
            elif ya == -1:
                actadj.append((la-1, 2, 1))
            elif ya == 2 and xa == 2 and y == 1:
                actadj.append((la+1, 0, 0))
                actadj.append((la+1, 1, 0))
                actadj.append((la+1, 2, 0))
                actadj.append((la+1, 3, 0))
                actadj.append((la+1, 4, 0))
            elif ya == 2 and xa == 2 and y == 3:
                actadj.append((la+1, 0, 4))
                actadj.append((la+1, 1, 4))
                actadj.append((la+1, 2, 4))
                actadj.append((la+1, 3, 4))
                actadj.append((la+1, 4, 4))
            elif ya == 5:
                actadj.append((la-1, 2, 3))
            else:
                actadj.append(aa)
        #print('actadj', l, x, y, sorted(actadj))
        return sum(1 for aa in actadj if aa in bugs)

    nbugs = set()
    for lev in range(minlev - 1, maxlev + 2):
        for i in range(5):
            for j in range(5):
                if i==2 and j==2:
                    continue
                if (lev,i,j) in bugs and countadj(lev,i,j) == 1:
                    nbugs.add((lev,i,j))
                if (lev,i,j) not in bugs and countadj(lev,i,j) in (1,2):
                    nbugs.add((lev,i,j))
    return nbugs


with open("/Users/yoannponti/Projects/programming-exercises/advent-of-code/data/2019/day24.txt") as f:
    lines = [l.rstrip('\n') for l in f]

    grid = lines
    seengrid = set()
    while True:
        fgrid = ''.join(grid)
        if fgrid in seengrid:
            break
        seengrid.add(fgrid)
        grid = step(grid)
    print(sum(2 ** i for i, ch in enumerate(fgrid) if ch == '#'))

    bugs = set()
    for i in range(5):
        for j in range(5):
            if lines[i][j] == '#':
                bugs.add((0,i,j))
    for i in range(200):
        bugs = stepb(bugs)
    print(len(bugs))


In [ ]:
grid = 0
with open("/Users/yoannponti/Projects/programming-exercises/advent-of-code/data/2019/day24.txt") as f:
    i = 0
    for line in f:
        for c in line.strip():
            if c == "#":
                grid += 1 << i
            i += 1

seen = set()
while grid not in seen:
    seen.add(grid)

    new_grid = 0
    for i in range(25):
        count = 0
        if i >= 5:
            count += (grid >> (i - 5)) & 1
        
        if i < 20:
            count += (grid >> (i + 5)) & 1
        
        if i % 5:
            count += (grid >> (i - 1)) & 1
        
        if i % 5 != 4:
            count += (grid >> (i + 1)) & 1
        
        if count == 1 or (count == 2 and not ((grid >> i) & 1)):
            new_grid |= 1 << i
    grid = new_grid

print(grid)


__Part 2__

## Day 25

http://adventofcode.com/2019/day/25

In [ ]:
content = load_day(25)

__Part 1__

__Part 2__